# 【先生専用】R_libs.zip の作成

このノートブックは **先生が一度だけ実行** して `R_libs.zip` を作成するためのものです。
作成した zip ファイルと RDS ファイルを学生に配布してください。

---

## 学生実習の全体フロー

```
【先生：1回のみ】
このノートブックを実行
  → Seurat 等をインストール
  → R_libs.zip をダウンロード
  → 学生に配布（R_libs.zip + JIAsyno_CITEseq_T_share.rds）

【学生：毎セッション開始時】
Colab に 2 ファイルをアップロード
  ├── R_libs.zip         （パッケージ群）
  └── JIAsyno_CITEseq_T_share.rds  （解析データ）
  → single_cell_analysis_T_colab.ipynb のセル1〜2を実行
```

## 手順
1. カーネルを **R** に設定（「ランタイム」→「ランタイムのタイプを変更」→ R）
2. **セル1** を実行 → パッケージをインストール（**40〜60分**）
3. **セル2** を実行 → `R_libs.zip` を作成してダウンロード

> ⚠️ すでに `/content/R_libs/` にパッケージが入っている場合は **セル1を省略** してセル2へ進めてください。


---
## セル1：パッケージのインストール（40〜60分）

> ✅ `/content/R_libs/` が存在する（すでにインストール済み）場合はスキップしてセル2へ。


In [ ]:
# ============================================================
# ローカルに R パッケージをインストール
# ============================================================
R_LIBS_LOCAL <- "/content/R_libs"
dir.create(R_LIBS_LOCAL, showWarnings = FALSE, recursive = TRUE)
.libPaths(c(R_LIBS_LOCAL, .libPaths()))

cat("📦 パッケージをインストール中...（40〜60分かかります）\n")

install_pkg <- function(pkg, bioc = FALSE) {
  if (!requireNamespace(pkg, quietly = TRUE, lib.loc = R_LIBS_LOCAL)) {
    cat("  ▶ インストール中:", pkg, "\n")
    tryCatch({
      if (bioc) {
        BiocManager::install(pkg, ask = FALSE, update = FALSE, lib = R_LIBS_LOCAL)
      } else {
        install.packages(pkg, lib = R_LIBS_LOCAL, repos = "https://cloud.r-project.org")
      }
      cat("    ✅", pkg, "完了\n")
    }, error = function(e) cat("    ❌ エラー:", conditionMessage(e), "\n"))
  } else {
    cat("  ✓ 既存:", pkg, "\n")
  }
}

install_pkg("BiocManager")
library(BiocManager, lib.loc = R_LIBS_LOCAL)
install_pkg("BiocStyle", bioc = TRUE)
install_pkg("Seurat")
for (pkg in c("magrittr", "patchwork", "dplyr", "ggplot2")) install_pkg(pkg)

pkgs <- c("Seurat", "BiocStyle", "magrittr", "patchwork", "dplyr", "ggplot2")
ok <- sapply(pkgs, function(p) requireNamespace(p, quietly = TRUE, lib.loc = R_LIBS_LOCAL))
cat("\n--- インストール結果 ---\n")
for (i in seq_along(pkgs)) cat(ifelse(ok[i], "✅", "❌"), pkgs[i], "\n")
cat("\n📦 パッケージ数:", length(list.files(R_LIBS_LOCAL)), "\n")
system(paste("du -sh", R_LIBS_LOCAL))
cat("\nセル2に進んでください。\n")


---
## セル2：R_libs.zip を作成してダウンロード

`/content/R_libs/` を zip 圧縮し、Colab からローカルPCにダウンロードします。
このファイル（`R_libs.zip`）を学生に配布してください。

> ℹ️ zip 圧縮は `/content` 全体のサイズによって数分かかります。


In [ ]:
# ============================================================
# R_libs.zip を作成 → ダウンロード
# ============================================================
cat("📦 R_libs を zip 圧縮中...\n")
system("cd /content && zip -r R_libs.zip R_libs/ -q")

# サイズ確認
info <- file.info("/content/R_libs.zip")
cat("✅ R_libs.zip 作成完了！\n")
cat("   ファイルサイズ:", round(info$size / 1e6, 1), "MB\n\n")

# Colab の UI でダウンロード
cat("⬇️  ダウンロード中...\n")
system("python3 -c \"\
from google.colab import files;\
files.download('/content/R_libs.zip')\
\"")

cat("\n✅ ダウンロード完了！\n")
cat("━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━\n")
cat("配布ファイル一覧：\n")
cat("  1. R_libs.zip                        ← 今ダウンロードしたもの\n")
cat("  2. JIAsyno_CITEseq_T_share.rds       ← 共有フォルダからDL済みのもの\n")
cat("━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━\n")
cat("この2ファイルを学生に配布し、\n")
cat("single_cell_analysis_T_colab.ipynb の手順に従って実行してもらってください。\n")
